### Import packages and set paths

In [ ]:
import dask.dataframe as dd
import dask_gateway
import dask.distributed

import dotenv
import warnings
warnings.filterwarnings('ignore')

import datetime
from datetime import datetime, timezone
from distfit import distfit
import geopandas as gpd
import math
import matplotlib as mpl
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import numpy as np
import os
import scipy
import scipy.stats as stats
from shapely import ops,wkt
from shapely.geometry import MultiLineString,Polygon, LineString, Point, MultiLineString
from shapely.ops import cascaded_union, transform
import pandas as pd
import pickle
import pyarrow as pa
import pyproj
import xarray as xr

In [ ]:
#sets the path to load pre-processed ais data
folder_name = '2022_PoR'
path_name = 'abfs://ais/parquet/' + folder_name  

#sets the path to load other local data
current_directory = os.getcwd()
path = current_directory.split("\\02_Data_Processing\\02_AIS_data")[0]

### Loads the access token (we use a SAS-token to protect the data)

In [ ]:
# this is for environmental variables for secrets (needs python-dotenv)
# You can copy the  .env.example file and rename it to .env (one directory  up from the notebooks)
# 
%load_ext dotenv
# Load environment variables from the .env file 1 directory up
%dotenv -v

In [ ]:
# read the environment variable from the  .env file
sas_token = dotenv.dotenv_values()['AZURE_BLOB_SAS_TOKEN']

### Creation of the cluster with high worker memory

In [ ]:
gateway = dask_gateway.Gateway()
cluster_options = gateway.cluster_options()
cluster_options.worker_memory = 64
cluster = gateway.new_cluster(cluster_options)
cluster.scale(n=1)
cluster

In [ ]:
client = dask.distributed.Client(cluster)
client

### Geospatial and other input data

In [ ]:
knots = 0.514444444
wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:32631')
wgs_to_utm = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
utm_to_wgs = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform

In [ ]:
anchorage_areas = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\anchorage_areas.geojson")
separation_zones = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\separation_zones.geojson")
separation_boundaries = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\separation_boundaries.geojson")
turning_basins = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\turning_basins.geojson")
water = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\water.geojson")
coastline = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\coastline.geojson")
liquid_bulk_terminals = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\liquid_bulk_terminals.geojson")
container_terminals = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\container_terminals.geojson")
dry_bulk_terminals = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\dry_bulk_terminals.geojson")

harbour_basins = pickle.load(open(path+"\\00_Input_data\\01_Geospatial_data\\harbour_basins_PoR.pickle",'rb'))
berths = pickle.load(open(path+"\\00_Input_data\\01_Geospatial_data\\selected_berths_PoR.pickle",'rb'))
berths.Harbour_basin = ['Waalhaven' if str(name).find('Waalhaven')+1 else name for name in berths.Harbour_basin]
berths.loc[berths[berths.Harbour_basin == 'IJselhaven'].index,'Harbour_basin'] = 'IJsselhaven'
berths.loc[berths[berths.Harbour_basin == 'Scheur'].index,'Harbour_basin'] = 'Scheurkade'
berths.loc[berths[berths.index == 'NIEUWE MAAS HBR HOLLAND AMERIKAKADE'].index,'Harbour_basin'] = 'Holland Amerika Kade'
berths.loc[berths[(berths.Terminal == 'VOPAK')&(berths.Harbour_basin == 'Nieuwe Maas')].index,'Harbour_basin'] = 'VOPAK'
berths.loc[berths[(berths.Terminal == 'NESTE')&(berths.Harbour_basin == 'Nieuwe Maas')].index,'Harbour_basin'] = 'Neste'
berths.loc[berths[(berths.Terminal == 'KTM')&(berths.Harbour_basin == 'Nieuwe maas')].index,'Harbour_basin'] = 'Koole Kade'
berths.loc[berths[berths.Harbour_basin == '3e Petroleumhaven'].index,'Harbour_basin'] = 'Botlek'
berths.loc[berths[berths.Harbour_basin == 'Torontohaven'].index,'Harbour_basin'] = 'Botlek'
berths.loc[berths[berths.Harbour_basin == 'Chemiehaven'].index,'Harbour_basin'] = 'Botlek'
berths.loc[berths[berths.Harbour_basin == '1e Werkhaven'].index,'Harbour_basin'] = 'Botlek'
berths.loc[berths[berths.Harbour_basin == '2e Werkhaven'].index,'Harbour_basin'] = 'Botlek'
berths.loc[berths[berths.Harbour_basin == 'Sint -Laurenshaven'].index,'Harbour_basin'] = 'Botlek'
berths.loc[berths[berths.Harbour_basin == 'Prins Willem-Alexanderhaven'].index,'Harbour_basin'] = 'Eemhaven'
berths.loc[berths[berths.Harbour_basin == 'Prins Johan Frisohaven'].index,'Harbour_basin'] = 'Eemhaven'
berths.loc[berths[berths.Harbour_basin == 'Prinses Beatrixhaven'].index,'Harbour_basin'] = 'Eemhaven'
berths = berths[berths.Harbour_basin != 'Koggehaven']
berths = berths[berths.index != 'OUDE MAAS HBR KADE']
berths = berths[berths.Harbour_basin != 'Zevenmanshaven']

harbour_basins.Geometry = harbour_basins.Geometry.apply(lambda x: transform(utm_to_wgs,x))
berths.geometry = berths.geometry.apply(lambda x: transform(utm_to_wgs,x))
turning_basins.geometry = turning_basins.geometry.apply(lambda x: transform(utm_to_wgs,x))
harbour_basins = harbour_basins.rename(columns={'Geometry':'geometry'})
turning_basins.index = turning_basins.index.astype(str)

maasvlakte_area = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\Maasvlakte_area.geojson")
botlek_area = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\Botlek_area.geojson")
europoort_area = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\Europoort_area.geojson")
vondelingenplaat_area = gpd.read_file(path+"\\00_Input_data\\01_Geospatial_data\\Vondelingenplaat_area.geojson")

#Prepare for origin/destination
anchorage_areas['name'] = anchorage_areas['seamark:name']
anchorage_areas = anchorage_areas.set_index('name')
anchorage_areas = anchorage_areas[['geometry']]

separation_zones['geometry'] = [Polygon(geom) for geom in separation_zones['geometry']] 
anchorage_areas['geometry'] = [Polygon(geom) for geom in anchorage_areas['geometry']] 
turning_basins['geometry'] = [Polygon(geom) for geom in turning_basins['geometry']] 

Koole = berths[(berths.Terminal == 'KOOLE')&(berths.Harbour_basin == '3e Petroleumhaven')]

In [ ]:
#List of fit distribution names
distr = ['beta',
         'chi2',
         'expon',
         'exponweib',
         'exponpow',
         'chi',
         'erlang',
         'f',
         'invgamma',
         'invgauss',
         'invweibull',
         'lognorm',
         'lomax',
         'powerlognorm',
         'recipinvgauss',
         'uniform',
         'gengamma',
         'halfnorm',
         'genexpon',
         'gausshyper',
         'genpareto',
         'gamma',
         'genhalflogistic',
         'halflogistic',
         'pareto',
         'rayleigh',
         'reciprocal',
         'truncexpon']

In [ ]:
#Some default colors
color_calandlijn = (226/255,33/255,18/255)
color_erasmuslijn = (2/255,58/255,141/255)
color_lijnA = (0/255,152/255,51/255)
color_lijnB = (253/255,223/255,1/255)
color_lijnC = (231/255,33/255,24/255)
color_lijnD = (52/255,180/255,229/255)
color_lijnE = (2/255,58/255,141/255)

water_color = 'lightblue'
boundary_color = 'k'
liquid_bulk_color = 'dodgerblue'
container_color = 'blue'
dry_bulk_color = 'midnightblue'
terminal_color = 'lightgrey'

In [ ]:
#Vessel type dataframe
type_dataframe = pd.DataFrame([120,150,200,230,285],columns=['length'],index=['Coaster','Handysize','Handymax','Panamax','Suezmax'])

### Open ship dataframe

In [ ]:
ddf = dd.read_parquet(path_name+'/ship_dataframe',storage_options={"account_name": "rwsais", "sas_token": sas_token})
ship_dataframe = ddf.compute()

### Open and prepare to voyage dataframe

In [ ]:
def renumber_index(df):
    """ 
    Function that sets the index as a column and resets the index with a new order

    Parameters
    ----------
    df: pandas dataframe with AIS data

    :returns: pandas dataframe
    """
    
    df = df.reset_index(drop=True)
    return df

def convert_string_geometry_to_shapely_geometry(df,geometry_columns):
    """ 
    Function that converts string geometry data to shapely geometries

    Parameters
    ----------
    df: pandas dataframe with trips
    geometry_columns: columns with geometry types as string data

    :returns: pandas dataframe
    """
    
    if df.empty or df['origin'].iloc[0] == 'a':
        return df
    for column in geometry_columns:
        geometries = []
        for loc,geom in df[column].items():
            if isinstance(geom,str) and geom != 'nan':
                geom = wkt.loads(geom)
            geometries.append(geom)
                
        df[column] = geometries
    return df

In [ ]:
ddf = dd.read_parquet(path_name+'/voyage_dataframe',storage_options={"account_name": "rwsais", "sas_token": sas_token})
geometry_columns = ['geometry_anchorage','geometry_entry','geometry_departure','coordinates']
ddf_i = ddf.partitions[:]
ddf_i = ddf_i.map_partitions(renumber_index)
ddf_i = ddf_i.map_partitions(convert_string_geometry_to_shapely_geometry,geometry_columns)
voyage_dataframe = ddf_i.compute()
voyage_dataframe = voyage_dataframe.rename(columns={'trip_number':'trip_id'})
voyage_dataframe = voyage_dataframe[voyage_dataframe.berth_of_call.isin(berths.index)]
voyage_dataframe = voyage_dataframe.reset_index(drop=True)

### Plots

In [ ]:
#Functions
def fit_distribution_to_data(x,data,axis,color='k',cutoffx = None, cutoffy = None,orientation='vertical',linestyle='--'):
    """ 
    Function that fits distributions to the data and selects the best fit and plots it

    Parameters
    ----------
    x: x data
    data: y data
    axis: matplotlib axis
    color: color
    cutoffx: cut-off in x
    cutoffy: cut-off in y
    orientation: orientation of the plot ('vertical' or 'horizontal')
    linestyle: matlotlib linestyle

    Returns
    -------
    obj: matplotlib object
    label: distribution name
    """
    
    dist = distfit(distr=distr);
    dist.fit_transform(np.array(data),verbose=50);
    obj = 0
    len_data = 1
    for _,dist in dist.summary.iterrows():
        if isinstance(dist['name'],float):
            continue
        distribution = getattr(stats,dist['name'])
        if not isinstance(cutoffx,float) and not isinstance(cutoffx,int):
            if orientation =='vertical':
                label = dist['name'] + ' (RSS: '+str(np.round(dist['score'],2))+')'
                y = distribution.pdf(x, *dist['params'])
                obj, = axis.plot(x,np.array(y)*len_data,color=color,label=label,linestyle=linestyle)
            elif orientation == 'horizontal':
                label = dist['name'] + ' (RSS: '+str(np.round(dist['score'],2))+')'
                y = distribution.pdf(x, *dist['params'])
                obj, = axis.plot(np.array(y)*len_data,x,color=color,label=label,linestyle=linestyle)
            break
            
        if distribution.pdf(cutoffx, *dist['params']) < cutoffy:
            if orientation =='vertical':
                label = dist['name'] + ' (RSS: '+str(np.round(dist['score'],2))+')'
                y = distribution.pdf(x, *dist['params'])
                obj, = axis.plot(x,np.array(y)*len_data,color=color,label=label,linestyle=linestyle)                
            elif orientation =='horizontal':
                label = dist['name'] + ' (RSS: '+str(np.round(dist['score'],2))+')'
                y = distribution.pdf(x, *dist['params'])
                obj, = axis.plot(np.array(y)*len_data,x,color=color,label=label,linestyle=linestyle)
            break
    if not obj:
        return
    else:
        return obj,label

def calculate_time_difference(df,parameter1,parameter2):
    """ 
    Function that calculates the difference in time between two parameters

    Parameters
    ----------
    df: pandas dataframe of voyages
    parameter1: column name of first variable to compare
    parameter2: column name of second variable to compare

    :returns: list of time differences
    """
    deltatimes = []
    for parameter1,parameter2 in zip(df[parameter1],df[parameter2]):
        if type(parameter1) == type(pd.Timestamp(2019,1,1)) and type(parameter2) == type(pd.Timestamp(2019,1,1)):
            deltatime = (parameter1-parameter2).total_seconds()/3600
        else:
            deltatime = 0
        deltatimes.append(deltatime)
    return deltatimes

def make_rgb_transparent(rgb, bg_rgb,alpha):
    """ 
    Function that creates a non-transparent color based on a transparent color
    
    Parameters
    ----------
    rgb: RGB-code as a list of RGB numbers from 0.0 to 1.0 as floats
    bg_rgb: background RGB-code as a list of RGB numbers from 0.0 to 1.0 as floats
    alpha: transparency of the RGB-color on the background RGB-color from 0.0 to 1.0 as float
    
    :returns: non-transparent color
    """
    
    non_transparent_color = [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]
    
    return non_transparent_color

In [ ]:
#Define shiptype dataframe
type_dataframe = pd.DataFrame({'Length':[120,150,180,294,366,400],
                               'Width':[22,24.5,29,32,49,77.5],
                               'Draught':[8.6,9.9,11.2,12,15.2,20.1],
                               'Heigth':[25,30,35,58,58,68],
                               'DWT':[24,35,50,80,100,200]},
                               index=['Coaster','Handysize','Handymax','Panamax','New Panamax','Suezmax'])

## Figure 04

In [ ]:
fig = plt.figure(figsize=[32,32])
gs = fig.add_gridspec(2,4)
axes = [fig.add_subplot(gs[0, :]),fig.add_subplot(gs[1,:3]),fig.add_subplot(gs[1,3])]
water_color = 'lightblue'
boundary_color = 'k'
anchorage_color = 'limegreen'
seperation_color = 'violet'
terminal_color = 'darkgrey'

ax = axes[0]
lon_min = 2.9
lon_max = 4.385
lat_min = 51.835
lat_max = 52.25
waterways=True
separation=True
liquids=True
containers=True
dries=True
turning_basin=False
anchorage_area=True
ax.set_facecolor('lightgrey')
if waterways:
    water.plot(ax=ax,facecolor=water_color,edgecolor='none',linewidth=2,zorder=97)
    ax.fill([0,0,0,0],[0,0,0,0],color=water_color,label='water')
    ax.fill([0,0,0,0],[0,0,0,0],facecolor='lightgrey',edgecolor='lightgrey',linewidth=5,label='land')
    ax.fill([0,0,0,0],[0,0,0,0],facecolor=terminal_color,edgecolor=terminal_color,linewidth=5,label='terminal area')

if separation:
    separation_zones.plot(ax=ax,facecolor=seperation_color,edgecolor='none',linewidth=5,linestyle='--',zorder=98)
    separation_boundaries.plot(ax=ax,facecolor="none",edgecolor=seperation_color,linewidth=5,linestyle='--',zorder=98)
    ax.fill([0,0,0,0],[0,0,0,0],facecolor='none',edgecolor=seperation_color,linestyle='--',linewidth=5,label='entrance channel')
    ax.fill([0,0,0,0],[0,0,0,0],color=seperation_color,label='separation zone')
    
if anchorage_area:
    anchorage_areas.plot(ax=ax,facecolor=water_color,edgecolor=anchorage_color,linewidth=5,linestyle='--',zorder=98)
    ax.fill([0,0,0,0],[0,0,0,0],facecolor='none',edgecolor=anchorage_color,linestyle='--',linewidth=5,label='anchorage area')

if liquids:
    liquid_bulk = liquid_bulk_terminals.reset_index(drop=True)
    new_liquid_bulk = liquid_bulk.copy()
    for loc,info in liquid_bulk.iterrows():
        new_liquid_bulk.loc[loc,'geometry'] = info['geometry'].simplify(tolerance=0.001).buffer(0.1, join_style=1).buffer(-0.1, join_style=1)
    new_liquid_bulk.plot(ax=ax,facecolor=terminal_color,edgecolor=terminal_color,linewidth=5)

if containers:
    container = container_terminals.reset_index(drop=True)
    new_container = container.copy()
    for loc,info in container.iterrows():
        new_container.loc[loc,'geometry'] = info['geometry'].simplify(tolerance=0.001).buffer(0.1, join_style=1).buffer(-0.1, join_style=1)
    new_container.plot(ax=ax,facecolor=terminal_color,edgecolor=terminal_color,linewidth=5)
    ax.fill([0,0,0,0],[0,0,0,0],facecolor="none",edgecolor='k',linewidth=3,label='berth')

if dries:
    dry_bulk = dry_bulk_terminals.reset_index(drop=True)
    new_dry_bulk = dry_bulk.copy()
    for loc,info in dry_bulk.iterrows():
        new_dry_bulk.loc[loc,'geometry'] = info['geometry'].simplify(tolerance=0.001).buffer(0.1, join_style=1).buffer(-0.1, join_style=1)
    new_dry_bulk.plot(ax=ax,facecolor=terminal_color,edgecolor=terminal_color,linewidth=5)

if turning_basin:
    turning_basins.plot(ax=ax,facecolor='none',edgecolor='k',linewidth=1,linestyle='--')
ax.plot([0,0], [0,0], marker='$\u25CC$', markerfacecolor='k', markeredgecolor='none', markersize=45, linestyle='none',label='turning basin')

#ax.legend(loc='upper right',bbox_to_anchor=[1.005,1.0125],prop={'size': 30}).set_zorder(102)
for spine in ax.spines.values():
    spine.set_visible(False)
ax.tick_params(bottom=False, labelbottom=False,
               left=False, labelleft=False)

for index in voyage_dataframe[~(voyage_dataframe.geometry_anchorage == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_anchorage.xy,color=color_calandlijn,linewidth=2,alpha=0.05,zorder=101)
for index in voyage_dataframe[~(voyage_dataframe.geometry_entry == 'nan') & ~(voyage_dataframe.geometry_departure == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_entry.xy,color=color_calandlijn,linewidth=2,alpha=0.05,zorder=100)
    ax.plot(*df_trip.geometry_departure.xy,color=color_erasmuslijn,linewidth=2,alpha=0.05,zorder=100)
for index in voyage_dataframe[~(voyage_dataframe.geometry_entry == 'nan') & (voyage_dataframe.geometry_departure == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_entry.xy,color=color_calandlijn,linewidth=2,alpha=0.05,zorder=100)
for index in voyage_dataframe[(voyage_dataframe.geometry_entry == 'nan') & ~(voyage_dataframe.geometry_departure == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_departure.xy,color=color_erasmuslijn,linewidth=2,alpha=0.05,zorder=100)

ax.set_xlim([lon_min,lon_max])
ax.set_ylim([lat_min,lat_max]);

# #################################################################################################################

ax = axes[1]
lon_min = 3.9
lon_max = 4.385
lat_min = 51.835
lat_max = 52.015
waterways=True
separation=True
liquids=True
containers=True
dries=True
turning_basin=True
anchorage_area=True
ax.set_facecolor('lightgrey')
if waterways:
    water.plot(ax=ax,facecolor=water_color,edgecolor='none',linewidth=2,zorder=100)
    ax.fill([0,0,0,0],[0,0,0,0],color=water_color,label='water')
    ax.fill([0,0,0,0],[0,0,0,0],facecolor='lightgrey',edgecolor='lightgrey',linewidth=5,label='land')
    ax.fill([0,0,0,0],[0,0,0,0],facecolor=terminal_color,edgecolor=terminal_color,linewidth=5,label='terminal area')

if separation:
    separation_zones.plot(ax=ax,facecolor=seperation_color,edgecolor='none',linewidth=5,linestyle='--',zorder=98)
    separation_boundaries.plot(ax=ax,facecolor="none",edgecolor=seperation_color,linewidth=5,linestyle='--',zorder=98)
    ax.fill([0,0,0,0],[0,0,0,0],facecolor='none',edgecolor=seperation_color,linestyle='--',linewidth=5,label='entrance channel')
    ax.fill([0,0,0,0],[0,0,0,0],color=seperation_color,label='separation zone')
    
if anchorage_area:
    anchorage_areas.plot(ax=ax,facecolor=water_color,edgecolor=anchorage_color,linewidth=5,linestyle='--',zorder=98)
    ax.fill([0,0,0,0],[0,0,0,0],facecolor='none',edgecolor=anchorage_color,linestyle='--',linewidth=5,label='anchorage area')

if liquids:
    liquid_bulk = liquid_bulk_terminals.reset_index(drop=True)
    new_liquid_bulk = liquid_bulk.copy()
    for loc,info in liquid_bulk.iterrows():
        new_liquid_bulk.loc[loc,'geometry'] = info['geometry'].simplify(tolerance=0.001).buffer(0.1, join_style=1).buffer(-0.1, join_style=1)
    new_liquid_bulk.plot(ax=ax,facecolor=terminal_color,edgecolor=terminal_color,linewidth=5)

if containers:
    container = container_terminals.reset_index(drop=True)
    new_container = container.copy()
    for loc,info in container.iterrows():
        new_container.loc[loc,'geometry'] = info['geometry'].simplify(tolerance=0.001).buffer(0.1, join_style=1).buffer(-0.1, join_style=1)
    new_container.plot(ax=ax,facecolor=terminal_color,edgecolor=terminal_color,linewidth=5)
    ax.fill([0,0,0,0],[0,0,0,0],facecolor="none",edgecolor='k',linewidth=3,label='berth')

if dries:
    dry_bulk = dry_bulk_terminals.reset_index(drop=True)
    new_dry_bulk = dry_bulk.copy()
    for loc,info in dry_bulk.iterrows():
        new_dry_bulk.loc[loc,'geometry'] = info['geometry'].simplify(tolerance=0.001).buffer(0.1, join_style=1).buffer(-0.1, join_style=1)
    new_dry_bulk.plot(ax=ax,facecolor=terminal_color,edgecolor=terminal_color,linewidth=5)

if turning_basin:
    turning_basins.plot(ax=ax,facecolor='none',edgecolor='k',linewidth=1,linestyle='--',zorder=101)
ax.plot([0,0], [0,0], marker='$\u25CC$', markerfacecolor='k', markeredgecolor='none', markersize=45, linestyle='none',label='turning basin')

#ax.legend(loc='upper right',bbox_to_anchor=[1.005,1.0125],prop={'size': 30}).set_zorder(102)
for spine in ax.spines.values():
    spine.set_visible(False)
ax.tick_params(bottom=False, labelbottom=False,
               left=False, labelleft=False)

for index in voyage_dataframe[~(voyage_dataframe.geometry_anchorage == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_anchorage.xy,color=color_calandlijn,linewidth=2,alpha=0.05,zorder=100)
for index in voyage_dataframe[~(voyage_dataframe.geometry_entry == 'nan') & ~(voyage_dataframe.geometry_departure == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_entry.xy,color=color_calandlijn,linewidth=2,alpha=0.05,zorder=100)
    ax.plot(*df_trip.geometry_departure.xy,color=color_erasmuslijn,linewidth=2,alpha=0.05,zorder=100)
for index in voyage_dataframe[~(voyage_dataframe.geometry_entry == 'nan') & (voyage_dataframe.geometry_departure == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_entry.xy,color=color_calandlijn,linewidth=2,alpha=0.05,zorder=100)
for index in voyage_dataframe[(voyage_dataframe.geometry_entry == 'nan') & ~(voyage_dataframe.geometry_departure == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_departure.xy,color=color_erasmuslijn,linewidth=2,alpha=0.05,zorder=100)
    
ax.set_xlim([lon_min,lon_max])
ax.set_ylim([lat_min,lat_max]);

ax.set_xlim([lon_min,lon_max])
ax.set_ylim([lat_min,lat_max])
for loc,info in maasvlakte_area.iloc[1:-1].iterrows():
    ax.fill(*info.geometry.exterior.coords.xy,facecolor=(0,0,0,0.1),edgecolor=(192/255,163/255,215/255),zorder=99,linewidth=3)
ax.fill(*europoort_area.iloc[0].geometry.exterior.coords.xy,facecolor=(0,0,0,0.1),edgecolor=(192/255,163/255,215/255),zorder=99,linewidth=3)
ax.fill(*botlek_area.iloc[0].geometry.exterior.coords.xy,facecolor=(0,0,0,0.1),edgecolor=(192/255,163/255,215/255),zorder=99,linewidth=3)
ax.fill(*vondelingenplaat_area.iloc[0].geometry.exterior.coords.xy,facecolor=(0,0,0,0.1),edgecolor=(192/255,163/255,215/255),zorder=99,linewidth=3);

####################################################################################################

ax = axes[2]
lon_min = 4.2925
lon_max = 4.32
lat_min = 51.869
lat_max = 51.9
waterways=True
separation=True
liquids=True
containers=True
dries=True
turning_basin=True
anchorage_area=True

ax.set_facecolor('lightgrey')
if waterways:
    water.plot(ax=ax,facecolor=water_color,edgecolor='none',linewidth=2,zorder=97)
    ax.fill([0,0,0,0],[0,0,0,0],color=water_color,label='water')
    ax.fill([0,0,0,0],[0,0,0,0],facecolor='lightgrey',edgecolor='lightgrey',linewidth=5,label='land')
    ax.fill([0,0,0,0],[0,0,0,0],facecolor=terminal_color,edgecolor=terminal_color,linewidth=5,label='terminal area')

if separation:
    separation_zones.plot(ax=ax,facecolor=seperation_color,edgecolor='none',linewidth=5,linestyle='--',zorder=98)
    separation_boundaries.plot(ax=ax,facecolor="none",edgecolor=seperation_color,linewidth=5,linestyle='--',zorder=98)
    ax.fill([0,0,0,0],[0,0,0,0],facecolor='none',edgecolor=seperation_color,linestyle='--',linewidth=5,label='entrance channel')
    ax.fill([0,0,0,0],[0,0,0,0],color=seperation_color,label='separation zone')
    
if anchorage_area:
    anchorage_areas.plot(ax=ax,facecolor=water_color,edgecolor=anchorage_color,linewidth=5,linestyle='--',zorder=98)
    ax.fill([0,0,0,0],[0,0,0,0],facecolor='none',edgecolor=anchorage_color,linestyle='--',linewidth=5,label='anchorage area')

if liquids:
    liquid_bulk = liquid_bulk_terminals.reset_index(drop=True)
    new_liquid_bulk = liquid_bulk.copy()
    for loc,info in liquid_bulk.iterrows():
        new_liquid_bulk.loc[loc,'geometry'] = info['geometry']
    new_liquid_bulk.plot(ax=ax,facecolor=terminal_color,edgecolor=terminal_color,linewidth=5)

if containers:
    container = container_terminals.reset_index(drop=True)
    new_container = container.copy()
    for loc,info in container.iterrows():
        new_container.loc[loc,'geometry'] = info['geometry']
    new_container.plot(ax=ax,facecolor=terminal_color,edgecolor=terminal_color,linewidth=5)
    ax.fill([0,0,0,0],[0,0,0,0],facecolor="none",edgecolor='k',linewidth=3,label='berth')

if dries:
    dry_bulk = dry_bulk_terminals.reset_index(drop=True)
    new_dry_bulk = dry_bulk.copy()
    for loc,info in dry_bulk.iterrows():
        new_dry_bulk.loc[loc,'geometry'] = info['geometry']
    new_dry_bulk.plot(ax=ax,facecolor=terminal_color,edgecolor=terminal_color,linewidth=5)

if turning_basin:
    turning_basins.plot(ax=ax,facecolor='none',edgecolor='k',linewidth=3,linestyle='--',zorder=99)
ax.plot([0,0], [0,0], marker='$\u25CC$', markerfacecolor='k', markeredgecolor='none', markersize=45, linestyle='none',label='turning basin')

Koole = berths[berths.index.isin(['Koole_Kade_H', 'Koole_Kade11', 'Koole_Buiten9', 'Koole_Buiten10', 'Koole_Kade_G', 'Koole_Binnen10'])]
Koole.plot(ax=ax,facecolor="none",edgecolor='k',linewidth=3,zorder=98)

#ax.legend(loc='upper right',bbox_to_anchor=[1.005,1.0125],prop={'size': 30}).set_zorder(102)
for spine in ax.spines.values():
    spine.set_visible(False)
ax.tick_params(bottom=False, labelbottom=False,
               left=False, labelleft=False)

for index in voyage_dataframe[~(voyage_dataframe.geometry_anchorage == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_anchorage.xy,color=color_calandlijn,linewidth=2,alpha=0.05,zorder=101)
for index in voyage_dataframe[~(voyage_dataframe.geometry_entry == 'nan') & ~(voyage_dataframe.geometry_departure == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_entry.xy,color=color_calandlijn,linewidth=2,alpha=0.05,zorder=100)
    ax.plot(*df_trip.geometry_departure.xy,color=color_erasmuslijn,linewidth=2,alpha=0.05,zorder=100)
for index in voyage_dataframe[~(voyage_dataframe.geometry_entry == 'nan') & (voyage_dataframe.geometry_departure == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_entry.xy,color=color_calandlijn,linewidth=2,alpha=0.05,zorder=100)
for index in voyage_dataframe[(voyage_dataframe.geometry_entry == 'nan') & ~(voyage_dataframe.geometry_departure == 'nan')].index:
    df_trip = voyage_dataframe.loc[index]
    ax.plot(*df_trip.geometry_departure.xy,color=color_erasmuslijn,linewidth=2,alpha=0.05,zorder=100)
    
ax.set_xlim([lon_min,lon_max])
ax.set_ylim([lat_min,lat_max]);
plt.subplots_adjust(wspace=0.02, hspace=-0.15);

ax = axes[0]
ax.plot([0,0],[0,0],color=color_calandlijn,linewidth=4,label='Inbound traffic')
ax.plot([0,0],[0,0],color=color_erasmuslijn,linewidth=4,label='Outbound traffic')
legend_entities,labels = ax.get_legend_handles_labels()
ax.legend([mpl.patches.Patch(facecolor=water_color,edgecolor=water_color,linewidth=5),
            mpl.patches.Patch(facecolor=make_rgb_transparent(mpl.colors.to_rgb('lightgrey'),mpl.colors.to_rgb('k'),alpha=0.9),edgecolor=(192/255,163/255,215/255),linewidth=5),
            legend_entities[3],
            legend_entities[5],
            legend_entities[6],
            legend_entities[8],
            legend_entities[9],
            legend_entities[1],
            mpl.patches.Patch(facecolor=terminal_color,edgecolor=terminal_color,linewidth=5),
            mpl.patches.Patch(facecolor='violet',edgecolor='violet',linewidth=5),
            mpl.patches.Patch(facecolor='none',edgecolor='none',linewidth=5),
            legend_entities[7],
            legend_entities[8],
            legend_entities[9]],
           ['','','','','','','','water / land','port / terminal area','separation zone','anchorage area','berth / turning basin','Inbound traffic','Outbound traffic'],
           handletextpad=0.5, handlelength=1.25, columnspacing=-0.5,
           loc='upper right',ncol=2,bbox_to_anchor=[1.005,1.0125],prop={'size': 30}).set_zorder(102)

legend = ax.get_legend()
legend.set_zorder(100)

fig.savefig(current_directory+'\\04_Output_data\\02_Figures\\Figure_04_AIS_PoR.png', dpi=500,bbox_inches='tight',facecolor=fig.get_facecolor());

## Figure 05

In [ ]:
shiptypes = []
for loc,info in voyage_dataframe.iterrows():
    types_length = list(type_dataframe[type_dataframe.Length >= ship_dataframe.loc[info['name']].length].index)
    types_width = list(type_dataframe[type_dataframe.Width >= ship_dataframe.loc[info['name']].width].index)
    types_draught_in = list(type_dataframe[type_dataframe.Draught >= info.draught_at_arrival].index)
    types_draught_out = list(type_dataframe[type_dataframe.Draught >= info.draught_at_departure].index)
    types = list(set(types_length) & set(types_width) & set(types_draught_in) & set(types_draught_out))
    for shiptype in type_dataframe.index:
        if shiptype in types:
            break
    shiptypes.append(shiptype)

In [ ]:
voyage_dataframe['shiptype'] = shiptypes

In [ ]:
#Define inbound and outbound voyage dataframes
in_df = voyage_dataframe[voyage_dataframe.arrival_at_port_entrance.notna() & voyage_dataframe.arrival_at_harbour_entrance.notna()]
out_df = voyage_dataframe[voyage_dataframe.departure_from_port_entrance.notna() & voyage_dataframe.departure_from_harbour_entrance.notna()]

#Define voyage dataframes with waiting times and calculate the waiting times
waiting_df = voyage_dataframe[~voyage_dataframe.anchorage_at_arrival.isna()]
waiting_times = (waiting_df.departure_from_anchorage_at_arrival-waiting_df.arrival_at_anchorage_at_arrival).astype('timedelta64[s]')/np.timedelta64(1,'D')

#Calculate sailing time of in- and outbound vessels
sailing_time_data_in = ((in_df.arrival_at_harbour_entrance-in_df.arrival_at_port_entrance).astype('timedelta64[s]')/np.timedelta64(1,'m'))
sailing_time_data_out = ((out_df.departure_from_port_entrance-out_df.departure_from_harbour_entrance).astype('timedelta64[s]')/np.timedelta64(1,'m'))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=[8,20])
fontsize=12
axes = []
n = 4 # number of double-rows
m = 1 # number of columns
for i in range(n*m):   
    n = 4 # number of double-rows
    m = 1 # number of columns

    t = 0.91 # 1-t == top space 
    b = 0.1 # bottom space      (both in figure coordinates)

    msp = 0.25 # minor spacing
    sp = 0.65  # major spacing
    
    if i in [0,3]:
        if not i:
            t = 0.82 # 1-t == top space 
            b = 0.1 # bottom space  
            msp = 0.4
            sp = 0  # major spacing
            
        offs=(1+msp)*(t-b)/(2*n+n*msp+(n-1)*sp) # grid offset
        hspace = sp+msp+1 #height space per grid
        gso = GridSpec(n,m, bottom=b+offs, top=t, hspace=hspace)
        axes.append(fig.add_subplot(gso[i]))
        continue
    
    offs=(1+msp)*(t-b)/(2*n+n*msp+(n-1)*sp) # grid offset
    hspace = sp+msp+1 #height space per grid
    gso = GridSpec(n,m, bottom=b+offs, top=t, hspace=hspace)    
    gse = GridSpec(n,m, bottom=b, top=t-offs, hspace=hspace)
    axes.append(fig.add_subplot(gso[i]))
    axes.append(fig.add_subplot(gse[i]))
    
data_in = sailing_time_data_in
data_out = sailing_time_data_out
x = np.arange(35,150,0.1)

ax = axes[1]
hist_in = ax.hist([data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Coaster'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Handysize'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Handymax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Panamax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'New Panamax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Suezmax']],
                  color= [make_rgb_transparent(color_erasmuslijn,(1,1,1),1/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),2/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),4/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),6/7)],
                  edgecolor='k',bins=np.arange(40,150,10),density=False,zorder=100,)
ax.set_xticks(np.arange(35,150,10))
ax.set_xticklabels([])
ax.set_xlim(35,135)
ax.set_ylabel('Number\nof vessels',fontsize=fontsize)
ax.set_ylim(0,np.ceil(np.max(hist_in[0])/10)*10)
ax.set_yticks(np.arange(0,np.ceil(np.max(hist_in[0])/10)*10+30,20))
ax.set_yticklabels([int(num) for num in np.arange(0,np.ceil(np.max(hist_in[0])/10)*10+30,20)],fontsize=fontsize)
ax.set_facecolor('none')
ax.yaxis.set_label_coords(-0.075,0.5)
ax.set_title('Sailing time',fontname="Arial", fontweight="bold",fontsize=14)

ax = axes[1].twinx()
ax.set_zorder(-1)
ax.set_ylim(0,0.04)
ax.set_yticks(np.arange(0,0.05,0.01))
ax.set_yticklabels(np.arange(0,0.05,0.01),fontsize=fontsize)
ax.set_ylabel('Density',fontsize=fontsize)
ax.yaxis.set_label_coords(1.1,0.5)
fit_distribution_to_data(x,data_in[voyage_dataframe.loc[data_in.index].shiptype.isin(['Panamax','New Panamax','Suezmax'])],ax,make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7))
fit_distribution_to_data(x,data_in,ax,make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),linestyle='-.')
fit_handles_inbound,_ = ax.get_legend_handles_labels()


ax = axes[2]
hist_out = ax.hist([data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Coaster'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Handysize'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Handymax'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Panamax'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'New Panamax'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Suezmax']],
                   color= [make_rgb_transparent(color_erasmuslijn,(1,1,1),1/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),2/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),4/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),6/7)],
                   edgecolor='k',bins=np.arange(40,150,10),density=False,zorder=100,)

ax.set_xticks(np.arange(35,140,10))
ax.set_xticklabels(np.arange(40,150,10),fontsize=fontsize)
ax.set_xlim(35,135)
ax.set_ylabel('Number\nof vessels',fontsize=fontsize)
ax.set_ylim(0,np.ceil(np.max(hist_out[0])/10)*10)
ax.set_yticks(np.arange(0,np.ceil(np.max(hist_out[0])/10)*10+20,20))
ax.set_yticklabels([int(num) for num in np.arange(0,np.ceil(np.max(hist_out[0])/10)*10+20,20)],fontsize=fontsize)
ax.set_facecolor('none')
ax.yaxis.set_label_coords(-0.075,0.5)
ax.set_xlabel('Time [min]',fontsize=fontsize)

ax= axes[2].twinx()
ax.set_zorder(-1)
ax.set_ylim(0,0.08)
ax.set_yticklabels(np.arange(0,0.1,0.02),fontsize=fontsize)
ax.yaxis.set_label_coords(1.1,0.5)
ax.set_yticklabels(ax.get_yticklabels(),fontsize=fontsize)
ax.set_ylabel('Density',fontsize=fontsize)
fit_distribution_to_data(x,data_out[voyage_dataframe.loc[data_out.index].shiptype.isin(['Panamax','New Panamax','Suezmax'])],ax,make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7))
fit_distribution_to_data(x,data_out,ax,make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),linestyle='-.')
fit_handles_outbound,_ = ax.get_legend_handles_labels()

voyage_dataframe_turning_inbound = voyage_dataframe[voyage_dataframe.turning_basin_at_arrival.notna()]
voyage_dataframe_turning_outbound = voyage_dataframe[voyage_dataframe.turning_basin_at_departure.notna()]
data_in = turningtime_data_in = (voyage_dataframe_turning_inbound.departure_from_turning_basin.astype('datetime64[ns, UTC]')-voyage_dataframe_turning_inbound.arrival_at_turning_basin.astype('datetime64[ns, UTC]'))/np.timedelta64(1,'m')
data_out = turningtime_data_out = (voyage_dataframe_turning_outbound.departure_from_turning_basin.astype('datetime64[ns, UTC]')-voyage_dataframe_turning_outbound.arrival_at_turning_basin.astype('datetime64[ns, UTC]'))/np.timedelta64(1,'m')
data_in = data_in[data_in > 0]
data_out = data_out[data_out > 0]

x = np.arange(0,25,0.1)

ax = axes[3]
hist_in = ax.hist([data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Coaster'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Handysize'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Handymax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Panamax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'New Panamax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Suezmax']],
                  color= [make_rgb_transparent(color_erasmuslijn,(1,1,1),1/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),2/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),4/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7),
                          make_rgb_transparent(color_erasmuslijn,(1,1,1),6/7)],
                  edgecolor='k',bins=np.arange(1.25,26.25,2.5),density=False,zorder=100,)
ax.set_xticks(np.arange(0,27.5,2.5))
ax.set_xticklabels([])
ax.set_xlim(0,25)
ax.set_ylabel('Number\nof vessels',fontsize=fontsize)
ax.set_yticks(np.arange(0,np.ceil(np.max(hist_in[0])/10)*10+50,50))
ax.set_yticklabels([int(num) for num in np.arange(0,np.ceil(np.max(hist_in[0])/10)*10+50,50)],fontsize=fontsize)
ax.set_facecolor('none')
ax.set_title('Turning time',fontname="Arial", fontweight="bold",fontsize=14)

ax = axes[3].twinx()
ax.set_zorder(-1)
ax.set_ylim(0,0.4)
ax.set_ylabel('Density',fontsize=fontsize)
ax.yaxis.set_label_coords(1.1,0.5)
ax.set_yticklabels(ax.get_yticklabels(),fontsize=fontsize)
fit_distribution_to_data(x,data_in[voyage_dataframe.loc[data_in.index].shiptype.isin(['Panamax','New Panamax','Suezmax'])],ax,make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7))
fit_distribution_to_data(x,data_in,ax,make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),linestyle='-.')

ax = axes[4]
hist_out = ax.hist([data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Coaster'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Handysize'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Handymax'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Panamax'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'New Panamax'],
                    data_out[voyage_dataframe.loc[data_out.index].shiptype == 'Suezmax']],
                   color= [make_rgb_transparent(color_erasmuslijn,(1,1,1),1/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),2/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),4/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),6/7)],
                   edgecolor='k',bins=np.arange(1.25,26.25,2.5),density=False,zorder=100,)
ax.set_xticks(np.arange(0,27.5,2.5))
ax.set_xticklabels(np.arange(0,27.5,2.5),fontsize=fontsize)
ax.set_xlim(0,25)
ax.set_ylabel('Number\nof vessels',fontsize=fontsize)
ax.set_ylim(0,np.ceil(np.max(hist_out[0])/10)*10)
ax.set_yticks(np.arange(0,np.ceil(np.max(hist_in[0])/10)*10+50,50))
ax.set_yticklabels([int(num) for num in np.arange(0,np.ceil(np.max(hist_in[0])/10)*10+50,50)],fontsize=fontsize)
ax.set_facecolor('none')
ax.yaxis.set_label_coords(-0.075,0.5)
ax.set_xlabel('Time [min]',fontsize=fontsize)

ax= axes[4].twinx()
ax.set_zorder(-1)
ax.set_ylim(0,0.4)
ax.set_ylabel('Density',fontsize=fontsize)
ax.set_yticklabels(ax.get_yticklabels(),fontsize=fontsize)
ax.yaxis.set_label_coords(1.1,0.5)
fit_distribution_to_data(x,data_out,ax,make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),linestyle='-.')

laytime_data = (voyage_dataframe.departure_from_berth.astype('datetime64[ns, UTC]')-voyage_dataframe.arrival_at_berth.astype('datetime64[ns, UTC]'))/np.timedelta64(1,'D')
data_in = laytime_data[(laytime_data.notna()) & (laytime_data > 0.05)]
mixed_color = make_rgb_transparent(color_calandlijn,color_erasmuslijn,0.5)

ax = axes[5]
hist_in = ax.hist([data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Coaster'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Handysize'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Handymax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Panamax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'New Panamax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Suezmax']],
                   color= [make_rgb_transparent(color_erasmuslijn,(1,1,1),1/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),2/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),4/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),6/7)],
                   edgecolor='k',bins=np.arange(0.25,5.25,0.5),density=False,zorder=100,)
ax.set_xticks(np.arange(0,5.25,1))
ax.set_xticklabels([int(value) for value in np.arange(0,5.25,1)],fontsize=fontsize)
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax2 = ax.twinx()
ax.patch.set_facecolor('none')
ax.set_zorder(2)
ax2.set_zorder(1)
x = np.arange(0,5.05,0.05)
fit_distribution_to_data(x,data_in[voyage_dataframe.loc[data_in.index].shiptype.isin(['Panamax','New Panamax','Suezmax'])],ax2,make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7))
fit_distribution_to_data(x,data_in,ax2,make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),linestyle='-.')
ax2.set_ylim(0,0.8)
ax2.set_yticks(np.arange(0,1,0.2))
ax2.set_yticklabels([np.round(value,1) for value in np.arange(0,1,0.2)],fontsize=fontsize)
ax2.set_xlim(0,5)
ax.set_xlim(0,5)
ax.set_ylim([0,150])
ax.set_yticks(np.arange(0,200,50))
ax.set_yticklabels(np.arange(0,200,50),fontsize=fontsize)
ax.set_xlabel('Time [days]',fontsize=fontsize)
ax.set_ylabel('Number\nof vessels',fontsize=fontsize)
ax.yaxis.set_label_coords(-0.075,0.5)
#ax2.set_ylabel('Density',fontsize=fontsize)
#ax2.yaxis.set_label_coords(1.1,0.5)
fit_handles,_ = ax2.get_legend_handles_labels()
ax.set_title('Lay time at terminal',fontname="Arial", fontweight="bold",fontsize=14)

ax = axes[0]
data_in = waiting_times
hist_in = ax.hist([data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Coaster'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Handysize'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Handymax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Panamax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'New Panamax'],
                   data_in[voyage_dataframe.loc[data_in.index].shiptype == 'Suezmax']],
                   color= [make_rgb_transparent(color_erasmuslijn,(1,1,1),1/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),2/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),4/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7),
                           make_rgb_transparent(color_erasmuslijn,(1,1,1),6/7)],
                   edgecolor='k',bins=np.arange(0.25,5.25,0.5),density=False,zorder=100,)
ax.set_xticks(np.arange(0,5.25,1))
ax.set_xticklabels([int(value) for value in np.arange(0,5.25,1)],fontsize=fontsize)
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.set_xlim(0,5)
ax2 = ax.twinx()
ax.patch.set_facecolor('none')
ax.set_zorder(2)
ax2.set_zorder(1)
x = np.arange(0,5.5,0.1)
fit_distribution_to_data(x,data_in[voyage_dataframe.loc[data_in.index].shiptype.isin(['Panamax','New Panamax','Suezmax'])],ax2,make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7))
fit_distribution_to_data(x,data_in,ax2,make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7),linestyle='-.')
ax2.set_ylim(0,1.6)
ax2.set_yticks(np.arange(0,2.0,0.4))
ax2.set_yticklabels([np.round(value,1) for value in np.arange(0,2.0,0.4)],fontsize=fontsize)
ax.set_ylim([0,40])
ax.set_yticks(np.arange(0,50,10))
ax.set_xlabel('Time [days]',fontsize=fontsize)
ax.set_ylabel('Number\nof vessels',fontsize=fontsize)
ax.yaxis.set_label_coords(-0.075,0.5)
ax2.set_ylabel('Density',fontsize=fontsize)
ax2.yaxis.set_label_coords(1.1,0.5)
fit_handles,_ = ax2.get_legend_handles_labels()
ax.set_title('Lay time at anchorage',fontname="Arial", fontweight="bold",fontsize=14)

ax = axes[0]
p_coaster1 = Patch(facecolor=make_rgb_transparent(color_calandlijn,(1,1,1),1/7), edgecolor='black')
p_coaster2 = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),1/7), edgecolor='black')
p_handysize1 = Patch(facecolor=make_rgb_transparent(color_calandlijn,(1,1,1),2/7), edgecolor='black')
p_handysize2 = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),2/7), edgecolor='black')
p_handymax1 = Patch(facecolor=make_rgb_transparent(color_calandlijn,(1,1,1),3/7), edgecolor='black')
p_handymax2 = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7), edgecolor='black')
p_panamax1 = Patch(facecolor=make_rgb_transparent(color_calandlijn,(1,1,1),4/7), edgecolor='black')
p_panamax2 = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),4/7), edgecolor='black')   
p_new_panamax1 = Patch(facecolor=make_rgb_transparent(color_calandlijn,(1,1,1),5/7), edgecolor='black')
p_new_panamax2 = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7), edgecolor='black')   
p_suezmax1 = Patch(facecolor=make_rgb_transparent(color_calandlijn,(1,1,1),6/7), edgecolor='black')
p_suezmax2 = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),6/7), edgecolor='black')   
p_coaster = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),1/7), edgecolor='black')
p_handysize = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),2/7), edgecolor='black')
p_handymax = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),3/7), edgecolor='black')
p_panamax = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),4/7), edgecolor='black') 
p_new_panamax = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),5/7), edgecolor='black')
p_suezmax = Patch(facecolor=make_rgb_transparent(color_erasmuslijn,(1,1,1),6/7), edgecolor='black')
p_empty = Patch(facecolor='none',edgecolor='none')
legend_obj = ax.legend(handles=[p_coaster,  p_handysize,  p_handymax,  p_panamax,  p_new_panamax,  p_suezmax, fit_handles[1], fit_handles[0]],
                       labels=['Coaster', 'Handysize', 'Handymax', 'Panamax', 'New Panamax', 'Suezmax', 'Distribution','Distribution\n(\N{GREATER-THAN OR EQUAL TO}Panamax)'],
                       loc='upper right',bbox_to_anchor=[1.45,1.15],frameon=False,prop={'size':12})

axes[4].text(25*0.99,150*0.85,'outbound',horizontalalignment='right',fontsize=fontsize)
axes[3].text(25*0.99,150*0.85,'inbound',horizontalalignment='right',fontsize=fontsize)
axes[2].text(135*0.99,80*0.85,'outbound',horizontalalignment='right',fontsize=fontsize)
axes[1].text(135*0.99,80*0.85,'inbound',horizontalalignment='right',fontsize=fontsize)
axes[0].text(5*0.99,50*1.1,'a)',horizontalalignment='right',weight='bold',fontsize=14)
axes[1].text(135*0.99,80*1.1,'b)',horizontalalignment='right',weight='bold',fontsize=14)
axes[3].text(25*0.99,150*1.1,'c)',horizontalalignment='right',weight='bold',fontsize=14)
axes[5].text(5*0.99,150*1.1,'d)',horizontalalignment='right',weight='bold',fontsize=14)
axes[0].text(-0.42,0,' ', fontsize=12, transform=axes[0].transAxes);

In [ ]:
fig.savefig(current_directory+'\\04_Output_data\\02_Figures\\Figure_05_Calibration_results_AIS_data.eps', format='eps', dpi=1000,bbox_inches='tight',facecolor=fig.get_facecolor())